## Following this tutorial
https://www.tensorflow.org/get_started/get_started#the_computational_graph

In [1]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import numpy as np

### Constants

In [6]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [7]:
sess = tf.Session()
sess.run([node1, node2])

[3.0, 4.0]

### Basic Algebra

In [9]:
node3 = node1 + node2
print(node3)
print(sess.run(node3))

Tensor("add_2:0", shape=(), dtype=float32)
7.0


### Place Holder

In [28]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a + b
print(c)
print(sess.run(c, feed_dict={a:1, b:3}))

Tensor("add_6:0", dtype=float32)
4.0


### Variable Initialization

In [84]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [86]:
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(W))
print(sess.run(b))
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.30000001]
[-0.30000001]
[ 0.          0.30000001  0.60000002  0.90000004]


### Calculate Loss

In [87]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


### Assign value to variable

In [65]:
fixW = tf.assign(W, [-1])
fixb = tf.assign(b, [1])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


### Training

In [124]:
optimizer = tf.train.GradientDescentOptimizer(0.02)
train = optimizer.minimize(loss)
sess.run(init)
for i in range(500):
    sess.run(train, feed_dict={x: [1, 2, 3, 4], y: [0, -1, -2, -3]})   
    if i % 20 == 0:
        sse = sess.run(loss, feed_dict={x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
        print("iteration: {0}, loss: {1}".format(i, sse))

iteration: 0, loss: 4.039775371551514
iteration: 20, loss: 0.5884453654289246
iteration: 40, loss: 0.22313465178012848
iteration: 60, loss: 0.08461123704910278
iteration: 80, loss: 0.03208404406905174
iteration: 100, loss: 0.012166056782007217
iteration: 120, loss: 0.0046132756397128105
iteration: 140, loss: 0.0017493258928880095
iteration: 160, loss: 0.0006633322918787599
iteration: 180, loss: 0.00025153032038360834
iteration: 200, loss: 9.537827281747013e-05
iteration: 220, loss: 3.616779940784909e-05
iteration: 240, loss: 1.3715072782360949e-05
iteration: 260, loss: 5.2005434554303065e-06
iteration: 280, loss: 1.9718784187716665e-06
iteration: 300, loss: 7.476127734662441e-07
iteration: 320, loss: 2.8343669100650004e-07
iteration: 340, loss: 1.0746391865268379e-07
iteration: 360, loss: 4.076203197200812e-08
iteration: 380, loss: 1.542969130241545e-08
iteration: 400, loss: 5.851120477018412e-09
iteration: 420, loss: 2.214804339928378e-09
iteration: 440, loss: 8.377583071705885e-10
it

### tf.estimator

In [13]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp40alwu13', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb6d44112e8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp40alwu13/model.ckpt.
INFO:tensorflow:loss = 23.0, step = 1
INFO:tensorflow:global_step/sec: 618.749
INFO:tensorflow:loss = 0.123438, step = 101 (0.162 sec)
INFO:tensorflow:global_step/sec: 746.461
INFO:tensorflow:loss = 0.0810343, step = 201 (0.134 sec)
INFO:tensorflow:global_step/sec: 577.795
INFO:tensorflow:loss = 0.022

### Custom Estimator

In [14]:
# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b

  loss = tf.reduce_sum(tf.square(y - labels))
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplmm8dg_y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb6df226978>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmplmm8dg_y/model.ckpt.
INFO:tensorflow:loss = 114.541993787, step = 1
INFO:tensorflow:global_step/sec: 670.457
INFO:tensorflow:loss = 0.00362417958034, step = 101 (0.150 sec)
INFO:tensorflow:global_step/sec: 964.361
INFO:tensorflow:loss = 0.000351173278993, step = 201 (0.104 sec)
INFO:tensorflow:global_step/sec: 908.495
INF

### Add and tf.add, softmax, cross_entropy and softmax_cross_entropy_with_logits

In [121]:
y1 = tf.Variable(np.array([0.4, 0.2, 0.1]).astype("float32"))
y2 = tf.Variable(np.array([4, 2, 7]).astype("float32"))
y3 = tf.Variable(np.array([0.1, 0.2, 4]).astype("float32"))
xent = 0
xent += tf.nn.softmax_cross_entropy_with_logits(labels=y1,logits=y2)
# xent += tf.nn.softmax_cross_entropy_with_logits(labels=y2,logits=y3)
# xent += tf.nn.softmax_cross_entropy_with_logits(labels=y1,logits=y3)

softmax1 = tf.nn.softmax(y1)
softmax2 = tf.nn.softmax(y2)

xent1 = tf.reduce_mean(-tf.reduce_sum(softmax1 * tf.log(softmax2)))

summ = y1 + y2
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([y1, y2]))
    print(sess.run(xent))
    print(sess.run(xent1))

[array([ 0.40000001,  0.2       ,  0.1       ], dtype=float32), array([ 4.,  2.,  7.], dtype=float32)]
2.23849
2.82643


In [112]:
np.array([0,0,1,1]).astype("float32")

array([ 0.,  0.,  1.,  1.], dtype=float32)